In [1]:
!pip install pytorch_lightning

In [2]:
# data_dir = "/scratch-shared/TANJ0303/datasets/"

In [3]:
# !export HF_DATASETS_CACHE=data_dir

In [4]:
import sys
import os

def is_colab_env():
    return "google.colab" in sys.modules

def mount_google_drive(drive_dir="/content/drive/", repo_dir="MyDrive/repositories/deepfake-detection"):
    # mount google drive
    from google.colab import drive
    drive.mount(drive_dir)

    # change to correct working directory
    import os
    os.chdir(f"{drive_dir}{repo_dir}")
    print(os.listdir()) # verify content

def resolve_path(levels_deep=3):
    if is_colab_env():
        mount_google_drive()
        return ""
    else:
        # Get the directory of the current script
        current_dir = os.path.dirname(os.path.abspath('__file__'))

        # Construct the path to the parent directory
        for i in range(levels_deep):
            current_dir = os.path.dirname(current_dir)

        # Add the parent directory to sys.path
        sys.path.append(current_dir)
        print(sys.path)
        return current_dir

proj_dir = resolve_path()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
['src', '.git', 'playground', 'setup', 'README.md', 'hf_wdf.sh', 'freqnet_image.ipynb', 'reports', '__pycache__', 'faceforensics_download.py', 'environment.yml', 'run_jupyter.sh', 'config.py', 'dct_mean_real_fake.png', 'analysis', '.gitignore']


In [5]:
# import local config
import config

In [6]:
# import library dependencies
import numpy as np

In [7]:
# pytorch
import torch
import pytorch_lightning as L

In [8]:
# import local dependencies
from src.adapters.datasets.sida import SidADataModule
from src.models.resnet import ResNetClassifier

In [9]:
model_id = "dwt_resnet18"
model_checkpoint_dir = f"{proj_dir}/{config.CHECKPOINTS_DIR}/{model_id}"

In [10]:
from src.transforms.frequency import get_transforms
dwt_transforms = get_transforms("wavelet")

In [11]:
seed = config.SEED

# Set seeds for reproducibility
torch.manual_seed(seed)
np.random.seed(seed)

# Determine device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [12]:
batch_size = 32
num_workers = 2

# define datamodule
sida_data_module = SidADataModule(
    batch_size=batch_size,
    num_workers=num_workers,
    seed=seed,
    transforms=dwt_transforms,
    # dataset_cache_dir=data_dir
)

In [13]:
# define early stopper
early_stop_callback = L.callbacks.EarlyStopping(
    monitor="val_loss",       # metric to track
    patience=3,               # epochs to wait for improvement
    mode="min",               # "min" because we want val_loss to decrease
    verbose=True
)

In [14]:
# define ligntning checkpoint
best_loss_checkpoint = L.callbacks.ModelCheckpoint(
    monitor="val_loss",
    mode="min",
    save_top_k=1,
)

In [15]:
# define model
checkpoint = "lightning_logs/version_0/checkpoints/epoch=7-step=202880.ckpt"
deepfake_detector = ResNetClassifier.load_from_checkpoint(f"{model_checkpoint_dir[1:]}/{checkpoint}")

In [16]:
# define trainer
max_epochs = 100
trainer = L.Trainer(
    devices=1,
    callbacks=[early_stop_callback, best_loss_checkpoint],
    default_root_dir=model_checkpoint_dir,
    log_every_n_steps=100,
    profiler="simple", # track time taken
    max_epochs=max_epochs,
  )

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [17]:
# test dataset on unseen samples
trainer.test(deepfake_detector, datamodule=sida_data_module)

Resolving data files:   0%|          | 0/249 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/34 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/249 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/34 [00:00<?, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.34950000047683716    │
│         test_auc          │    0.47878867387771606    │
│          test_f1          │    0.13255099952220917    │
│         test_loss         │     1.423789620399475     │
│         test_prec         │    0.5971165299415588     │
│         test_rec          │    0.07455000281333923    │
└───────────────────────────┴───────────────────────────┘

INFO:pytorch_lightning.profilers.profiler:TEST Profiler Report

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                         	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                                                                                                                                       

[{'test_acc': 0.34950000047683716,
  'test_prec': 0.5971165299415588,
  'test_rec': 0.07455000281333923,
  'test_f1': 0.13255099952220917,
  'test_auc': 0.47878867387771606,
  'test_loss': 1.423789620399475}]